In [1]:
import pandas as pd
import numpy as np
import os
import re
import functools

In [2]:
pd.set_option('max_colwidth',250)
pd.set_option('display.width',250)

In [3]:
os.getcwd()

'/home/matt/Git/PGIP/sandbox'

In [4]:
os.chdir('..')
os.chdir('src')

In [5]:
import pipeline as PGIP

In [6]:
os.chdir('..')

In [7]:
E = PGIP.Explorer('config/cache')

In [8]:
P = PGIP.Pipeline('config/cache')

# Read sample Data

In [11]:
E.readSampleData('sandbox')

Data files: ['ISI_CCC-proteomics.csv', 'ISI_CCC-transcriptomics.csv']
column sample: ['PT' 'AU' 'AF' 'CA' 'TI' 'SO' 'SE' 'ED' 'LA' 'DT']
column sample: ['PT' 'AU' 'AF' 'CA' 'TI' 'SO' 'SE' 'ED' 'LA' 'DT']
Data files: ['EV-transcriptomics.xlsx', 'EV-proteomics.2013-2016.xlsx']
column sample: ['Title' 'Accession number' 'Author' 'Author affiliation'
 'Corresponding author' 'Source' 'Abbreviated source title' 'Sponsor'
 'Publisher' 'Volume']
column sample: ['Title' 'Accession number' 'Author' 'Author affiliation'
 'Corresponding author' 'Source' 'Abbreviated source title' 'Sponsor'
 'Publisher' 'Volume']
Data files: ['Scopus-gene synthesis.xlsx', 'Scopus-synthetic gene.xlsx', 'Scopus-synthetic gene network.xlsx']
column sample: ['Authors' 'Title' 'Year' 'Source title' 'Volume' 'Issue' 'Art. No.'
 'Page start' 'Page end' 'Page count']
column sample: ['Authors' 'Title' 'Year' 'Source title' 'Volume' 'Issue' 'Art. No.'
 'Page start' 'Page end' 'Page count']
column sample: ['Authors' 'Title' '

In [12]:
E.renameColumns()

In [138]:
def parseAuthorAffiliationsEV(row):
    aus = row.loc['Authors']
    afs = row.loc['Affiliations']
    aus = aus.split(';')
    afs = afs.split(';')

    inst = dict()
    auth = dict()
    df = pd.DataFrame(columns = ['Author','Institution'])
    
    for af in afs:
        n = re.findall('\([0-9]+\)',af)[0]        
        afClean = re.sub('\([0-9]+\)','',af).strip()

        afClean = re.split('[,\(\)]',afClean)

        remove = []
        for string in afClean:
            if len(string.strip()) < 8:
                remove = remove + [string]
                continue
            if len(re.findall('[a-zA-Z]+',string)) < 2:
                remove = remove + [string]
                continue
            if len(re.findall('[0-9]+',string)) > 0:
                remove = remove + [string]
                continue

        afClean = [string for string in afClean if string not in remove]
        inst[n] = afClean

    for au in aus:
        n = re.findall('\([,0-9]+\)',au)[0]
        n = re.findall('[0-9]',n)
        n = ['('+i+')' for i in n]
        auClean = re.sub('\([,0-9]+\)','',au).strip()
        
        for i in n:
            for ins in inst[i]:
                df = pd.concat([df,pd.DataFrame({'Author':[auClean],'Institution':[ins]})])
    
    return df
        

In [233]:
def getAbbrevs(data,column):
    strings = data[column].values
    abbrevs = []
    for string in strings:
        words = str(string).split()
        for word in words:
            if word.strip().endswith('.'):
                abbrevs = abbrevs + [word]
                
    abbrevs = pd.Series(abbrevs).unique()
    abbrevs = sorted(list(abbrevs))
    return abbrevs

In [234]:
abbreviations = dict()

In [235]:
for column in ['Authors','Affiliations','PublicationTitle']:
    abbreviations[column] = dict()
    for source in E.sources:
        s = E.data[source].data[[column]]
        abbrevs = getAbbrevs(s,column)
        abbreviations[column][source] = abbrevs 

In [238]:
abbreviations['PublicationTitle']

{'EV': [],
 'IEEE': ["'09.", '2009.'],
 'ISI_CCC': [],
 'ISI_Inspec': ['(BICOB-2013).',
  '2012.',
  '2013).',
  '2013.',
  '2014.',
  'Applications.',
  'Bioinformatics.',
  'Biology.',
  'Databases.',
  'Engineering.',
  'Intelligence.',
  'Technology.',
  'Theories.',
  'TrAC.',
  'World.'],
 'ISI_Medline': ['University.', 'bao.', 'cancer.', 'journal.', 'physiology.'],
 'ISI_WoS': [],
 'Scopus': ['A.S.', 'G.']}

In [237]:
import json
os.getcwd()
json.dump(abbreviations,open('config/cache/abbreviations.json','w'))

In [39]:
source = 'ISI_Inspec'
b = E.sampleColumns(source,['Authors','Affiliations','AuthorsWithAffiliations','PublicationDate'],10)
b

,Authors,Affiliations,AuthorsWithAffiliations,PublicationDate
0,Yinghai Lv; Xiaolu Su; Fuli Ren; Hong-qiang Kan; Ying Kong,NaN,"Yinghai Lv; Fuli Ren; Hong-qiang Kan; Coll. of Chem. & Environ. Eng., Shandong Univ. of Sci. & Technol., Qingdao, China.Xiaolu Su; Sch. of Med. Lab. Sci. & Life Sci., WenZhou Med. Coll., Wenzhou, China.Ying Kong; State Key Lab. of Heavy Oil Proce...",Aug.
1,Yang Wang; Rui Gao; Pei-Pei Wang; Jia Jian; Xian-Li Jiang; Chao Yan; Xiao Lin; Li Wu; Guo-Qiang Chen; Qiong Wu,NaN,"Yang Wang; Rui Gao; Pei-Pei Wang; Xian-Li Jiang; Chao Yan; Xiao Lin; Li Wu; Qiong Wu; Minist. of Educ. Key Lab. of Bioinf., Tsinghua Univ., Beijing, China.Jia Jian; Guo-Qiang Chen; Protein Sci. Lab. of the Minist. of Educ., Tsinghua Univ., Beijin...",Jan.
2,"Dawei Deng; Dongyin Zhang; Yang Li; Achilefu, S.; Yueqing Gu",NaN,"Dawei Deng; Dongyin Zhang; Yang Li; Yueqing Gu; Dept. of Biomed. Eng., China Pharm. Univ., Nanjing, China.Achilefu, S.; Dept. of Radiol., Washington Univ., St. Louis, MO, USA.",15 Nov.
3,"Gehrmann, T.; Loog, M.; Reinders, M.J.T.; de Ridder, D.",NaN,"Gehrmann, T.; Reinders, M.J.T.; de Ridder, D.; Delft Bioinf. Lab., Delft Univ. of Technol., Delft, Netherlands.Loog, M.; Pattern Recognition Lab., Delft Univ. of Technol., Delft, Netherlands.",2013
4,Li Wang; Yongqian Zhang; Hong Qing; Yajun Wang; Mei Zhang; Changgen Feng; Yulin Deng,NaN,"Li Wang; Yajun Wang; Changgen Feng; Sch. of Mechatron. Eng., Beijing Inst. of Technol., Beijing, China.Yongqian Zhang; Hong Qing; Mei Zhang; Yulin Deng; Sch. of Life Sci., Beijing Inst. of Technol., Beijing, China.",2012
5,Junshan Yang; Zexuan Zhu; Shan He; Zhen Ji,NaN,"Junshan Yang; Zexuan Zhu; Zhen Ji; Shenzhen City Key Lab. of Embedded Syst. Design Coll. of Comput. Sci. & Software Eng., Shenzhen Univ., Shenzhen, China.Shan He; Sch. of Comput. Sci., Univ. of Birmingham, Birmingham, UK.",2013
6,Xiuquan Du; Jiaxing Cheng; Tingting Zheng; Zheng Duan; Fulan Qian,NaN,"Xiuquan Du; Key Lab. of Intell. Comput. & Signal Process. of Minist. of Educ., Anhui Univ., Hefei, China.Jiaxing Cheng; Inst. of Inf. Eng., Anhui Xinhua Univ., Hefei, China.Zheng Duan; Fulan Qian; Sch. of Comput. Sci. & Technol., Anhui Univ., Hef...",July
7,Liang Qianjin; Wang Pengcheng; Bai Yanrong,NaN,"Liang Qianjin; Wang Pengcheng; Bai Yanrong; Beijing Key Lab. of Gene Eng. Drugs & Biol. Technol., Beijing Normal Univ., Beijing, China.",2012
8,"Jia Meng; Xiaodong Cui; Hui Liu; Lin Zhang; Shaowu Zhang; Rao, M.K.; Yidong Chen; Yufei Huang",NaN,"Jia Meng; Dept. of Biol. Sci., Xi'an Jiaotong-Liverpool Univ., Suzhou, China.Xiaodong Cui; Yufei Huang; Dept. of ECE, Univ. of Texas at San Antonio, San Antonio, TX, USA.Hui Liu; Lin Zhang; Sch. of Inf. & Electr. Eng., China Univ. of Min. & Techn...",2013
9,"Zhi-Ping Liu; Jiguang Wang; Yu-Qing Qiu; Leung, R.K.K.; Xiang-Sun Zhang; Tsui, S.K.W.; Luonan Chen",NaN,"Zhi-Ping Liu; Jiguang Wang; Luonan Chen; Key Lab. of Syst. Biol., Shanghai Inst. for Biol. Sci., Shanghai, China.Xiang-Sun Zhang; Nat. Center for Math. & Interdiscipl. Sci., Beijing, China.Yu-Qing Qiu; Acad. of Math. & Syst. Sci., Beijing, China....",2012


In [40]:
b['Source'] = source

In [41]:
[P.adders.addMonth(b.loc[i,:]) for i in range(0,b.shape[0])]

[8, 1, 11, nan, nan, nan, 7, nan, nan, nan]

In [ ]:
E.tryMap(P.cleaners.cleanPublicationDate,'PublicationDate','IEEE')